In [ ]:
# default_exp utils_blitz

# uitls_blitz

> API details.

In [ ]:
#export
#hide
from blitz.modules import BayesianLinear
from blitz.modules import BayesianEmbedding, BayesianConv1d, BayesianConv2d, BayesianConv3d
from blitz.modules.base_bayesian_module import BayesianModule
from torch import nn
import torch
from fastcore.basics import patch

In [ ]:
@patch
def extra_repr(self: BayesianLinear):
    return f"Shape: {list(self.weight_sampler.mu.shape)}"

@patch
def extra_repr(self: BayesianConv1d):
    return f"in_channels={self.in_channels}, out_channels={self.out_channels}, kernelel_size={self.kernel_size}, stride={self.stride}"

@patch
def extra_repr(self: BayesianConv2d):
    return f"in_channels={self.in_channels}, out_channels={self.out_channels}, kernelel_size={self.kernel_size}, stride={self.stride}"

@patch
def extra_repr(self: BayesianConv3d):
    return f"in_channels={self.in_channels}, out_channels={self.out_channels}, kernelel_size={self.kernel_size}, stride={self.stride}"


In [ ]:
#export
def convert_layer_to_bayesian(layer, config: dict):
    if isinstance(layer, torch.nn.Linear):
        new_layer = BayesianLinear(
            layer.in_features,
            layer.out_features,
            prior_sigma_1=config["prior_sigma_1"],
            prior_sigma_2=config["prior_sigma_2"],
            prior_pi=config["prior_pi"],
            posterior_mu_init=config["posterior_mu_init"],
            posterior_rho_init=config["posterior_rho_init"],
        )
    elif isinstance(layer, nn.Embedding):
        new_layer = BayesianEmbedding(
            layer.num_embeddings, 
            layer.embedding_dim,
            prior_sigma_1=config["prior_sigma_1"],
            prior_sigma_2=config["prior_sigma_2"],
            prior_pi=config["prior_pi"],
            posterior_mu_init=config["posterior_mu_init"],
            posterior_rho_init=config["posterior_rho_init"],
        )
    elif isinstance(layer, (nn.Conv1d, nn.Conv2d, nn.Conv3d)):
        matching_class = BayesianConv1d
        kernel_size = layer.kernel_size[0]
        if type(layer) == nn.Conv2d:
            kernel_size = layer.kernel_size
            matching_class = BayesianConv2d
        elif type(layer) == nn.Conv3d:
            matching_class = BayesianConv3d
            kernel_size = layer.kernel_size
        new_layer = matching_class(
            layer.in_channels,
            layer.out_channels,
            kernel_size=kernel_size,
            groups=layer.groups,
            padding=layer.padding,
            dilation=layer.dilation,
            prior_sigma_1=config["prior_sigma_1"],
            prior_sigma_2=config["prior_sigma_2"],
            prior_pi=config["prior_pi"],
            posterior_mu_init=config["posterior_mu_init"],
            posterior_rho_init=config["posterior_rho_init"],
        )
    else:
        Warning(
            f"Could not find correct type for conversion of layer {layer} with type {type(layer)}"
        )
        new_layer = layer

    return new_layer

In [ ]:
config = {"prior_sigma_1":0.1,
            "prior_sigma_2":0.4,
            "prior_pi":1,
            "posterior_mu_init":0,
            "posterior_rho_init":-7}

In [ ]:
convert_layer_to_bayesian(nn.Linear(10,2), config)

BayesianLinear(
  Shape: [2, 10]
  (weight_sampler): TrainableRandomDistribution()
  (bias_sampler): TrainableRandomDistribution()
  (weight_prior_dist): PriorWeightDistribution()
  (bias_prior_dist): PriorWeightDistribution()
)

In [ ]:
convert_layer_to_bayesian(nn.Embedding(10,2), config)

BayesianEmbedding(
  (weight_sampler): TrainableRandomDistribution()
  (weight_prior_dist): PriorWeightDistribution()
)

In [ ]:
convert_layer_to_bayesian(nn.Conv1d(1,2,3), config)

BayesianConv1d(
  in_channels=1, out_channels=2, kernelel_size=3, stride=1
  (weight_sampler): TrainableRandomDistribution()
  (bias_sampler): TrainableRandomDistribution()
  (weight_prior_dist): PriorWeightDistribution()
  (bias_prior_dist): PriorWeightDistribution()
)

In [ ]:
convert_layer_to_bayesian(nn.Conv2d(1,2,(3,3)), config)

BayesianConv2d(
  in_channels=1, out_channels=2, kernelel_size=(3, 3), stride=1
  (weight_sampler): TrainableRandomDistribution()
  (bias_sampler): TrainableRandomDistribution()
  (weight_prior_dist): PriorWeightDistribution()
  (bias_prior_dist): PriorWeightDistribution()
)

In [ ]:
convert_layer_to_bayesian(nn.Conv2d(1,2,(3,3, 3)), config)

BayesianConv2d(
  in_channels=1, out_channels=2, kernelel_size=(3, 3, 3), stride=1
  (weight_sampler): TrainableRandomDistribution()
  (bias_sampler): TrainableRandomDistribution()
  (weight_prior_dist): PriorWeightDistribution()
  (bias_prior_dist): PriorWeightDistribution()
)

In [ ]:
#export
def convert_to_bayesian_model(model, config: dict):
    for p in model.named_children():
        cur_layer_name = p[0]
        cur_layer = p[1]
        if len(list(cur_layer.named_children())) > 0:
            convert_to_bayesian_model(cur_layer, config)
        elif not isinstance(cur_layer, BayesianModule):
            new_layer = convert_layer_to_bayesian(cur_layer, config)
            setattr(model, cur_layer_name, new_layer)

    return model


In [ ]:
convert_to_bayesian_model(nn.Sequential(nn.Linear(3,4), nn.Linear(4,1)), config)

Sequential(
  (0): BayesianLinear(
    Shape: [4, 3]
    (weight_sampler): TrainableRandomDistribution()
    (bias_sampler): TrainableRandomDistribution()
    (weight_prior_dist): PriorWeightDistribution()
    (bias_prior_dist): PriorWeightDistribution()
  )
  (1): BayesianLinear(
    Shape: [1, 4]
    (weight_sampler): TrainableRandomDistribution()
    (bias_sampler): TrainableRandomDistribution()
    (weight_prior_dist): PriorWeightDistribution()
    (bias_prior_dist): PriorWeightDistribution()
  )
)

In [ ]:
#export
def set_train_mode(model, mode):
    if isinstance(model, BayesianModule):
        model.freeze = not mode

    for module in model.children():
        set_train_mode(module, mode)